## **Extraction des PDFS**

In [1]:
!pip install -q requests beautifulsoup4 tqdm


In [6]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from tqdm import tqdm

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Academic ESILV Scraper)"
}

session = requests.Session()
session.headers.update(HEADERS)


In [7]:
RUBRICS = {
    "admissions": [
        "https://www.esilv.fr/admissions/"
    ],
    "formations": [
        "https://www.esilv.fr/formations/"
    ],
    "international": [
        "https://www.esilv.fr/international/"
    ],
    "entreprises": [
        "https://www.esilv.fr/entreprises/"
    ],
    "recherche": [
        "https://www.esilv.fr/recherche/"
    ],
    "lecole": [
        "https://www.esilv.fr/lecole/"
    ]
}



In [8]:
def extract_pdf_links(page_url):
    pdfs = set()

    r = session.get(page_url, timeout=15)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    for a in soup.select("a[href]"):
        href = a["href"]
        link = urljoin(page_url, href)

        # PDFs explicites OU endpoints /download/
        if re.search(r"\.pdf(\?.*)?$", link, re.IGNORECASE) or "/download/" in link:
            pdfs.add(link)

    return pdfs


In [9]:
def sanitize_filename(url):
    name = urlparse(url).path.strip("/").replace("/", "_")
    if not name:
        name = "document"
    if not name.endswith(".pdf"):
        name += ".pdf"
    return name


def download_pdf(url, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    filename = sanitize_filename(url)
    path = os.path.join(out_dir, filename)

    if os.path.exists(path):
        return

    r = session.get(url, stream=True, timeout=30)
    r.raise_for_status()

    # Sécurité finale
    if "application/pdf" not in r.headers.get("Content-Type", "").lower():
        return

    with open(path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 64):
            if chunk:
                f.write(chunk)



In [10]:
all_results = {}

for rubric, pages in RUBRICS.items():
    print(f"\n📂 Rubrique : {rubric}")
    rubric_pdfs = set()

    for page in pages:
        rubric_pdfs |= extract_pdf_links(page)

    print(f"  ➜ PDFs trouvés : {len(rubric_pdfs)}")

    out_dir = f"pdfs/{rubric}"

    for pdf in tqdm(rubric_pdfs, desc=f"Téléchargement {rubric}"):
        download_pdf(pdf, out_dir)

    all_results[rubric] = list(rubric_pdfs)




📂 Rubrique : admissions
  ➜ PDFs trouvés : 10


Téléchargement admissions: 100%|██████████| 10/10 [00:21<00:00,  2.18s/it]



📂 Rubrique : formations
  ➜ PDFs trouvés : 10


Téléchargement formations: 100%|██████████| 10/10 [00:21<00:00,  2.14s/it]



📂 Rubrique : international
  ➜ PDFs trouvés : 10


...

Téléchargement entreprises: 100%|██████████| 11/11 [00:22<00:00,  2.04s/it]



📂 Rubrique : recherche
  ➜ PDFs trouvés : 10


Téléchargement recherche: 100%|██████████| 10/10 [00:21<00:00,  2.19s/it]



📂 Rubrique : lecole
  ➜ PDFs trouvés : 10


Téléchargement lecole: 100%|██████████| 10/10 [00:22<00:00,  2.23s/it]


## **Conversion en JSON**

In [11]:
!pip install -q pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.5 MB/s eta 0:00:00


In [12]:
import os
import json
import re
import pdfplumber

PDF_DIR = "pdfs/entreprises"
OUTPUT_JSON = "entreprises_pdfs.json"


In [13]:
def clean_text(text):
    text = re.sub(r"\s+", " ", text)   # espaces multiples
    text = text.replace("•", "-")      # puces
    return text.strip()


In [14]:
def extract_pdf(pdf_path):
    pages = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            raw = page.extract_text()
            if not raw:
                continue

            pages.append({
                "page": i,
                "text": clean_text(raw)
            })

    return pages


In [15]:
result = {
    "rubric": "entreprises",
    "documents": []
}

for pdf_file in sorted(os.listdir(PDF_DIR)):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(PDF_DIR, pdf_file)

    doc = {
        "pdf_name": pdf_file,
        "pages": extract_pdf(pdf_path)
    }

    result["documents"].append(doc)


In [16]:
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("✅ entreprises_pdfs.json généré")


✅ entreprises_pdfs.json généré
